In [1]:
import cv2
import numpy as np
import imutils
from collections import deque

In [2]:
#after testing hsv 
# greenLower = (29, 86, 6)
# greenUpper = (64, 255, 255)
# greenLower = (41, 30, 74)
# greenUpper = (86, 144, 255)
greenLower =(40, 25 , 0)
greenUpper =(86, 255, 255)
pts = deque(maxlen=20)

In [3]:
video = cv2.VideoCapture("ccc.mov")

fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out = cv2.VideoWriter('outputt.avi',fourcc, 25.0,(1000,562),True)

xmin = 1000
xmin2= 1000
flag = 1
flag3 = 1
final=None
# xxx="No Goal"
while(video.isOpened()):
    flag2= 0
    _, frame = video.read()
    if frame is None:
        break
#     frame = imutils.rotate(frame, 180)
    frame = imutils.resize(frame, width=1000)
    (h, w) = frame.shape[:2]
    
    
    
    
    #LINE detection  # detect xmin2

    if flag==1:
        frame2=frame.copy()
        hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
        #lower_white = np.array([0, 0, 212])
        #upper_white = np.array([131, 255, 255])
        #lower_white = np.array([80 ,41  ,0])
        #upper_white = np.array([113 ,255 ,255])
        lower_white = np.array([86  , 0 ,129])
        upper_white = np.array([195 , 18, 255])

        mask = cv2.inRange(hsv, lower_white, upper_white)

        kernel = np.ones((4,4), np.uint8)
        mask = cv2.erode(mask, kernel, iterations=1)
        mask = cv2.dilate(mask, kernel, iterations=1)
        #cv2.imshow("mask",mask)

        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        if len(contours) > 0:
            cv2.drawContours(frame2, [contours[0]], -1, (0, 255, 0), 5)
            for i in range (0,len(contours[0])):
                #print(contours[0][i][0][0])
                if contours[0][i][0][0] < xmin2:
                    xmin2=contours[0][i][0][0]
            cv2.line(frame2, (xmin2,0),(xmin2,h),(0,0,225),5)
        #cv2.imshow("frame2",frame2)
        flag=0
        
        
#     if flag3==1:
#         frame3=frame.copy()
#         hsv = cv2.cvtColor(frame3, cv2.COLOR_BGR2HSV)
#         #lower_white = np.array([0, 0, 212])
#         #upper_white = np.array([131, 255, 255])
#         #lower_white = np.array([80 ,41  ,0])
#         #upper_white = np.array([113 ,255 ,255])
#         lower_white = np.array([86  , 0 ,129])
#         upper_white = np.array([195 , 18, 255])

#         mask = cv2.inRange(hsv, lower_white, upper_white)

#         kernel = np.ones((4,4), np.uint8)
#         mask = cv2.erode(mask, kernel, iterations=1)
#         mask = cv2.dilate(mask, kernel, iterations=1)
#         cv2.imshow("mask",mask)

#         contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         contours = sorted(contours, key=cv2.contourArea, reverse=True)
#         if len(contours) > 0:
#             peri = cv2.arcLength(contours[0] , True)
#             approx = cv2.approxPolyDP(contours[0], 0.01 * peri, True)
#             # ensure that the approximated contour is "roughly" rectangular
#             print(len(approx))
#             if len(approx) >= 4:
#                 #print(approx[0][0])
#                 point1=(approx[0][0][0],approx[0][0][1])
#                 point2=(approx[1][0][0],approx[1][0][1])
#                 cv2.line(frame3,point1,point2,(255,0,0),5)
#         #cv2.imshow("frame3",frame3)
#         flag3=0
        
        
        
    
    
    
    
    
    
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    kernel=np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    mask = cv2.dilate(mask, kernel, iterations=2)
    mask = cv2.erode(mask, kernel, iterations=2)
    res=cv2.bitwise_and(frame,frame,mask=mask)
    cnts,_=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]
    #cv2.imshow("mask",mask)
    
    center = None
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        
        if x<xmin and x>50:
            xmin=x
            #final=frame.copy()
            flag2=1
        
        if radius > 70:
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
            if x+radius < xmin2:
                xxx="Goal"
            else:
                xxx="No Goal"

    if (xxx=="No Goal"):
        cv2.putText(frame, "No Goal", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 6)
    else:
        cv2.putText(frame, "Goal", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 6)
        
    cv2.line(frame, (xmin2,0),(xmin2,h),(0,0,225),5)
#     cv2.line(frame,point1,point2,(255,0,0),2)
    if flag2==1:
        final=frame.copy()
    cv2.imshow("Frame", frame)
    #(h, w) = frame.shape[:2]
    #print(h)
    #print(w)
    out.write(frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        break

if final is not None:
    cv2.imshow("extreme left",final)
    cv2.waitKey()
    cv2.imwrite("final.png",final)

video.release()
out.release()
cv2.destroyAllWindows()